## How do I make an RDD?

RDDs can be created from stable storage or by transforming other RDDs. Run the cells below to create RDDs from files on the local drive.  All data files can be downloaded from the course moodle.

----------

##  RDD operations

In [1]:
# Read data from local file system:
print(sc.version)

fruits = sc.textFile('data/fruits.txt')
yellowThings = sc.textFile('data/yellowthings.txt')
print(fruits.collect())
print(yellowThings.collect())

3.3.1


['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry']
['banana', 'bee', 'butter', 'canary melon', 'gold', 'lemon', 'pineapple', 'sunflower']


In [2]:
# map
fruitsReversed = fruits.map(lambda fruit: fruit[::-1])

In [3]:
#fruitsReversed = fruits.map(lambda fruit: fruit[::-1])
fruitsReversed.cache()
# try changing the file and re-execute with and without cache
print(fruitsReversed.collect())
# What happens when you uncomment the first line and run the whole program again with cache()?

['elppa', 'ananab', 'nolem yranac', 'parg', 'nomel', 'egnaro', 'elppaenip', 'yrrebwarts']


In [4]:
# filter
k = 5
shortFruits = fruits.filter(lambda fruit: len(fruit) <= k)
print(shortFruits.collect())

['apple', 'grap', 'lemon']


In [5]:
# flatMap
characters = fruits.flatMap(lambda fruit: list(fruit))
print(characters.collect())

['a', 'p', 'p', 'l', 'e', 'b', 'a', 'n', 'a', 'n', 'a', 'c', 'a', 'n', 'a', 'r', 'y', ' ', 'm', 'e', 'l', 'o', 'n', 'g', 'r', 'a', 'p', 'l', 'e', 'm', 'o', 'n', 'o', 'r', 'a', 'n', 'g', 'e', 'p', 'i', 'n', 'e', 'a', 'p', 'p', 'l', 'e', 's', 't', 'r', 'a', 'w', 'b', 'e', 'r', 'r', 'y']


In [6]:
# union
fruitsAndYellowThings = fruits.union(yellowThings)
print(fruitsAndYellowThings.collect())

['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry', 'banana', 'bee', 'butter', 'canary melon', 'gold', 'lemon', 'pineapple', 'sunflower']


In [7]:
# intersection
yellowFruits = fruits.intersection(yellowThings)
print(yellowFruits.collect())

['banana', 'pineapple', 'canary melon', 'lemon']


In [8]:
# distinct
distinctFruitsAndYellowThings = fruitsAndYellowThings.distinct()
print(distinctFruitsAndYellowThings.collect())

['banana', 'orange', 'pineapple', 'butter', 'gold', 'sunflower', 'apple', 'canary melon', 'grap', 'lemon', 'strawberry', 'bee']


### RDD actions
Following are examples of some of the common actions available. For a detailed list, see [RDD Actions](https://spark.apache.org/docs/2.0.0/programming-guide.html#actions).

Run some transformations below to understand this better. Place the cursor in the cell and press **SHIFT + ENTER**.

In [9]:
# collect
fruitsArray = fruits.collect()
yellowThingsArray = yellowThings.collect()
print(fruitsArray)

['apple', 'banana', 'canary melon', 'grap', 'lemon', 'orange', 'pineapple', 'strawberry']


In [10]:
# count
numFruits = fruits.count()
print(numFruits)

8


In [11]:
# take
first3Fruits = fruits.take(3)
print(first3Fruits)

['apple', 'banana', 'canary melon']


In [12]:
print(fruits.map(lambda fruit: len(fruit)).sum())
print(fruits.map(lambda fruit: len(fruit)).reduce(lambda x, y: x*y))

57
3888000


In [13]:
# reduce
letterSet = fruits.map(lambda fruit: set(fruit)).reduce(lambda x, y: x.union(y))
print(letterSet)

{'n', 'b', 'l', 'o', 'r', 'a', 'i', 'm', 'p', 't', 'g', 's', 'c', ' ', 'y', 'e', 'w'}


In [14]:
letterSet = fruits.flatMap(lambda fruit: list(fruit)).distinct().collect()
print(letterSet)

['a', 'p', 'l', 'e', 'b', 'n', 'c', 'r', 'y', ' ', 'm', 'o', 'g', 'i', 's', 't', 'w']


### Key-Value Pairs

In [15]:
# reduceByKey
numFruitsByLength = fruits.map(lambda fruit: (len(fruit), 1)).reduceByKey(lambda x, y: x + y)
print(numFruitsByLength.take(10))

[(5, 2), (6, 2), (12, 1), (4, 1), (9, 1), (10, 1)]


In [16]:
from operator import add

lines = sc.textFile('data/course.txt')
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)
print(counts.sortByKey().take(20))

[('Big', 1), ('Course', 2), ('Data', 1), ('Description', 1), ('Information', 1), ('Management', 1), ('To', 3), ('and', 1), ('application', 1), ('big', 1), ('case', 1), ('conduct', 1), ('data', 4), ('fundamental', 1), ('how', 1), ('introduce', 1), ('issues', 1), ('large-scale', 1), ('latest', 1), ('learn', 1)]


In [17]:
print(counts.sortBy(lambda x: x[1], False).take(20))

[('data', 4), ('To', 3), ('Course', 2), ('the', 2), ('of', 2), ('techniques', 2), ('management', 2), ('Information', 1), ('Big', 1), ('Data', 1), ('Management', 1), ('Description', 1), ('introduce', 1), ('fundamental', 1), ('issues', 1), ('big', 1), ('management;', 1), ('learn', 1), ('latest', 1), ('and', 1)]


In [18]:
# Join simple example

products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
#trans = sc.parallelize([(1, 134, "OK"), (3, 34, "OK"), (5, 162, "Error"), (1, 135, "OK"), (2, 53, "OK"), (1, 45, "OK")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print(products.join(trans).take(20))

[(2, ('Orange', (53, 'OK'))), (1, ('Apple', (134, 'OK'))), (1, ('Apple', (135, 'OK'))), (1, ('Apple', (45, 'OK'))), (3, ('TV', (34, 'OK'))), (5, ('Computer', (162, 'Error')))]


### Join vs. Broadcast Variables

In [20]:
products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print(trans.join(products).take(20))


[(2, ((53, 'OK'), 'Orange')), (1, ((134, 'OK'), 'Apple')), (1, ((135, 'OK'), 'Apple')), (1, ((45, 'OK'), 'Apple')), (3, ((34, 'OK'), 'TV')), (5, ((162, 'Error'), 'Computer'))]


In [21]:
products = {1: "Apple", 2: "Orange", 3: "TV", 5: "Computer"}
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

broadcasted_products = sc.broadcast(products)

results = trans.map(lambda x: (x[0], broadcasted_products.value[x[0]], x[1]))
#  results = trans.map(lambda x: (x[0], products[x[0]], x[1]))
print(results.take(20))


[(1, 'Apple', (134, 'OK')), (3, 'TV', (34, 'OK')), (5, 'Computer', (162, 'Error')), (1, 'Apple', (135, 'OK')), (2, 'Orange', (53, 'OK')), (1, 'Apple', (45, 'OK'))]


In [23]:
sc.stop()